<img src="https://www.ifsc.edu.br/image/layout_set_logo?img_id=1319584&t=1602803233260" width="20%">

<center>

---
# **Análise de Sentimento Multilíngue usando BERT**
## <u>Prof. Carlos Andres Ferrero</u>
## Instituto Federal de Santa Catarina (IFSC), Câmpus Lages
## Grupo de Pesquisa em Análise Inteligente de Dados (IDA-IFSC)
---
</center>

# Introdução

**Objetivo**: introduzir o uso do modelo BERT para construir um modelo de classificação multilíngue para análise de sentimento de *reviews* de aplicativos.

**Material e Método**: 

*Etapa 1 - Coleta de Dados:* os dados utilizados para contruir o modelo de classificação correspondem a avaliações de aplicativos, os quais foram extraídos utilizando webscrapping no site da Google Play Store. Cada avaliação possuia inicialmente um score de 1 a 5 e esses scores foram transformados em três classes ou labels: `postivo (score>=3)`, `negativo (score<3)`.

*Etapa 2 - Pré-processamento de Dados:* o atributo classe foi codificado e as avaliações tokenizadas (transformação em *tokens*) usando BertTokenizer. Nesta etapa são apresentados e explicados alguns conceitos e parâmetros usados na tokenização.

*Etapa 3 - Preparação dos Conjuntos de Dados:* o conjunto de dados com todas as avaliações é dividido em três conjuntos: treinamento, validação e teste.

*Etapa 4 - Definição do Modelo de Classificação:* um modelo para classificação de sentenças baseado em BERT é apresentado, bem como uma função para transformar as instâncias do problema em entradas (`input`) e saídas (`output`) corretos para o treinamento do modelo em questão.

*Etapa 5 - Treinamento do Modelo*: os conjuntos de dados treinamento e validação são utilizados para treinar o modelo, monitorar o treinamento e escolher o melhor modelo.

*Etapa 6 - Avaliação do Modelo*: os conjunto de teste é utilizado para avaliar o modelo de classificação com dados não observados durante o treinamento ou o monitoramento do treinamento. O modelo foi avaliado utilizando medida de acurácia.

**Escopo**:

O escopo deste documento é limitado a estudar análise de sentimento por meio de classificação, usando um dos modelos do estado da arte em Processamento de Linguagem Natural (NLP) e abstrai os conceitos de WordEmbeddings, Sentence Embeddings e estruturas internas do modelo de Rede Neural para classificação: como Recurrent Neural Networks e Attention Layer.

# Desenvolvimento

Instalação e importação de bibliotecas.

In [1]:
!pip install transformers

In [1]:
# Basic
import csv
import pandas as pd
import numpy as np
import os
import re

# Tensorflow
import tensorflow as tf

# BERT
from transformers import BertTokenizer, BasicTokenizer
from transformers import TFBertModel, TFBertPreTrainedModel, TFBertForSequenceClassification, BertConfig

# Scikit-leran
from sklearn.model_selection import train_test_split

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## Etapa 1 - Coleta de Dados

Carregando os dois conjuntos de dados de avaliações em inlgês e português.

In [2]:
df_en = pd.read_csv('../data/app_reviews_en_processed.csv')
df_pt = pd.read_csv('../data/app_reviews_pt_processed.csv')
df = pd.concat([df_en,df_pt], ignore_index=True).dropna()
df

,text,label,score
0,Not enjoying the app yet.. Paid for full acces...,negative,1
1,This app implied that it had better integratio...,negative,1
2,The trial period is too short to really know i...,negative,1
3,"I've used this before and dumped it, and I gav...",negative,1
4,"Used to be great, am now stuck with this bug t...",negative,1
...,...,...,...
28306,Muito bom,positive,5
28307,"Bastante funcional, reune em um só app varias ...",positive,5
28308,Melhor aplicativo da linha!,positive,5
28309,Muito util,positive,5


Selecionar avaliações positivas e negativas

In [3]:
df.label.value_counts()

positive    19456
negative     8854
Name: label, dtype: int64

In [4]:
df.label.value_counts(normalize=True).round(2)

positive    0.69
negative    0.31
Name: label, dtype: float64

In [5]:
columns = ['text','label']
df = df[columns] 

## Etapa 2 - Pré-processamento de Dados

### Codificação do atributo classe

In [6]:
to_replace = {
    'negative' : 0,    
    'positive' : 1    
}

df['label'].replace(to_replace, inplace=True)

df

d:\Users\andres\git_projects\multi-language-sentiment-analysis\env\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,text,label
0,Not enjoying the app yet.. Paid for full acces...,0
1,This app implied that it had better integratio...,0
2,The trial period is too short to really know i...,0
3,"I've used this before and dumped it, and I gav...",0
4,"Used to be great, am now stuck with this bug t...",0
...,...,...
28306,Muito bom,1
28307,"Bastante funcional, reune em um só app varias ...",1
28308,Melhor aplicativo da linha!,1
28309,Muito util,1


### Tokenização de Sentenças usando BertTokenizer

In [7]:
PRETRAINED_MODEL = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL, do_lower_case=False)

In [8]:
sentence = "Not enjoying the app yet. Paid for full access for a year, and the WhatsApp feature isn't working properly"
sentence

"Not enjoying the app yet. Paid for full access for a year, and the WhatsApp feature isn't working properly"

In [9]:
tokenized_sentence = tokenizer(sentence)

for k, v in tokenized_sentence.items():
    print(k,v)

input_ids [101, 16040, 84874, 10230, 10105, 72894, 21833, 119, 107353, 10162, 10142, 13375, 18314, 10142, 169, 10924, 117, 10111, 10105, 12489, 10107, 10738, 16587, 19072, 98370, 112, 188, 14616, 83438, 102]
token_type_ids [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


O tokenizador BertTokenizer transforma cada sentença em três arrays de valores: `input_ids`, `token_type_ids` e `attention_mask`.
- `input_ids`: correspondem a índices de um dicionário (comumente denominado `vocab`) a que cada temo da sentença corresponde. Adicionalmente Bert adiciona alguns `ids` especiais, como de inicio e fim de sentença, bem como pontuações. Nesta versão do Bert, `bert-base-multilingual-cased`, temos um amplo vocabulário que inclui termos em mais de 100 idiomas e na forma case sensitive (palavras com maiúsculas ou minúsculas possuem `ids` diferentes).
- `token_type_ids`: em algumas aplicações Bert é utilizado em diálogos de pergunta resposta, como bots para conversa ou avaliação. No nosso estudo usamos apenas um sentença, portanto cada toke corresponde à mesma sentença.
- `attention_mask`: existem sentenças com poucos termos e outras com muitos termos. Em geral é necessário definir um tamanho máximo de termos para codificar e analisar, por exemplo os primeiros 10 tokens. Assim, sentenças com mais de 10 tokens são truncadas e, sentenças com menos, são preenchidas (`padding`) com `ids=0`. A informação de `attention_mask` indica quais `ids` devem ser considerados e quais não.

No exemplo abaixo, realizamos utilizandos as opções de número máximo de tokens para 10, utilizando as opções de `truncation` e `padding`:

In [10]:
sentences = ["WhatsApp feature isn't working properly","This is the best App","Not so bad"]
tokenized_sentences = tokenizer(sentences, max_length=10, padding=True, truncation=True)

for k, v in tokenized_sentences.items():
    print(k,v)

input_ids [[101, 12489, 10107, 10738, 16587, 19072, 98370, 112, 188, 102], [101, 10747, 10124, 10105, 12504, 73784, 102, 0, 0, 0], [101, 16040, 10380, 15838, 102, 0, 0, 0, 0, 0]]
token_type_ids [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]


Decodificando as sentenças pelos seus `input_ids`:

In [11]:
for i in range(len(sentences)):
    decoded = tokenizer.decode(tokenized_sentences['input_ids'][i])
    print("Original: {} | Decoded: {}".format(sentences[i], decoded) )    

Original: WhatsApp feature isn't working properly | Decoded: [CLS] WhatsApp feature isn't [SEP]
Original: This is the best App | Decoded: [CLS] This is the best App [SEP] [PAD] [PAD] [PAD]
Original: Not so bad | Decoded: [CLS] Not so bad [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]


Note que:
- Na sentença temos também token especiais que definem o inicio \[CLS\] e fim \[SEP\] da sentença, bem como de preenchimento \[PAD\].
- A primeira sentença foi truncada porque a trasformação em tokens resultou em mais do que o limite permitido de tokens.
- Nas sentenças 2 e 3 foi necessário realizar padding para preencher o espaço até 10 termos. Consequentmente os seus respectivos arrays `attention_mask` representam a informação de em quais tokens deve ter atenção e quais não.
As configurações de `max_length`, `padding` e `truncation`, são muito utilizadas no processo de tokenização de sentenças.

A seguir é apresentada uma função para codificar as nossas sentenças com configurações específicas.

In [12]:
def encode_sentences(X, tokenizer, max_length):
    return tokenizer.batch_encode_plus(X,
        max_length=max_length, # tamanho máximo da sequencia de tokens
        truncation=True, # truncar a sentença
        padding=True, # usar padding
        add_special_tokens=True, # adicionar tokens especiais [CLS] e [SEP]
        return_attention_mask=True, # Retornar attention_mask
        return_token_type_ids=False, # NÃO retornar token_type_ids, pois não são necessários        
        return_tensors='tf' # Formato para usar TensorFlow/Keras
        )

Execução da função `encode_sentences` para as três primeiras sentenças 

In [13]:
sentences = df.text[:3]
print(sentences)

0    Not enjoying the app yet.. Paid for full acces...
1    This app implied that it had better integratio...
2    The trial period is too short to really know i...
Name: text, dtype: object


In [14]:
MAX_SEQUENCE_LENGTH = 64
encode_sentences(sentences, tokenizer, MAX_SEQUENCE_LENGTH )

{'input_ids': <tf.Tensor: shape=(3, 64), dtype=int32, numpy=
array([[   101,  16040,  84874,  10230,  10105,  72894,  21833,    119,
           119, 107353,  10162,  10142,  13375,  18314,  10142,    169,
         10924,    117,  10111,  10105,  12489,  10107,  10738,  16587,
         19072,  98370,    112,    188,  14616,  83438,    117,  27156,
         29421,    169,  11639,  14956,  11304,  10142,  30767,    117,
         21001,  13383,  12014,  22807,  11639,  14956,  11304,  10374,
         10590,  11847,  10142,  24848,    119,  10747,  10124,  10472,
         10105,  10422,  34469,    119,  10747,  14819,  10347,    102],
       [   101,  10747,  72894,  10211, 104309,  10189,  10271,  10374,
         18322,  64861,  10169,  41181,  20999,  11084,  10271,  10106,
         18638,  15107,    119,  10576,  17895,    117,  10271,  10124,
           169,  14772,  11989,    117,  13800,    118,  10105,  23582,
         10108,  11178,  23599,  10114,  11847,  11639,  14956,  25779,
  

## Etapa 3 - Preparação dos Conjuntos de Dados

A função `train_val_test_split` abaixo é uma extensão de `train_test_split` com suporte a treino, validação e teste.

In [15]:
from sklearn.model_selection import train_test_split

def train_val_test_split(X, y, val_size = 0.25, test_size = 0.25, random_state = 42, shuffle = True, stratify = None ):    
    X_train, X_aux, y_train, y_aux = \
        train_test_split(X, y, test_size = val_size + test_size, random_state=random_state, shuffle=shuffle, stratify=stratify)
    X_val, X_test, y_val, y_test = \
        train_test_split(X_aux, y_aux, test_size = test_size / (val_size + test_size), random_state=random_state, shuffle=shuffle,stratify=y_aux if stratify is not None else None)
    return X_train, X_val, X_test, y_train, y_val, y_test

A divisão do conjunto de dados em treino, validação e teste, ocorre da seguinte forma:
- `train` é usado para treinar o modelo de classificação
- `val` é usado como validação do modelo com novos durante o treinamento, a cada época. Isso permite monitorar o desempenho do algoritmo com novos dados, evitar possível overfitting, treinamento desnecessário e escolher um modelo com melhor desempenho para avaliação no futuro.
- `test` é usado, ao final, para avaliar o modelo escolhido com o conjunto de validação.

Neste trabalho será utilizada uma distribuição de 20\% para treino 40\% para validação e 40\% para teste.

In [16]:
from sklearn.model_selection import train_test_split
X = df['text']
y = df['label']

val_size = 0.25
test_size = 0.25
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, val_size=val_size, test_size=test_size, stratify=y)

In [17]:
train_sentences_tokenized = tokenizer.batch_encode_plus(X_train)

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


In [18]:
train_sentence_lengths = [ len(x) for x in train_sentences_tokenized['input_ids']]
print('90% das sentenças tem até {:.0f} tokens'.format( np.percentile(train_sentence_lengths, 90) ) )

90% das sentenças tem até 102 tokens


In [19]:
MAX_SEQUENCE_LENGTH = int( np.percentile(train_sentence_lengths, 90) )
MAX_SEQUENCE_LENGTH

102

In [20]:
num_labels = len(y_train.unique()) # número de classes do problema

## Etapa 4 - Definição do Modelo de Classificação

Instanciamos o modelo BERT para classificação de sequências de tokens.

In [27]:
model = TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=num_labels, output_attentions=False, output_hidden_states=False)
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 177,854,978
Trainable params: 177,854,978
Non-trainable params: 0
_________________________________________________________________


O modelo de classificação BERT recebe como entrada um array com dois elementos. O primeiro elemento são os `input_ids` das sequências e o segundo elemento é a máscara de atenção (`attention_mask`) de cada sentença. 

Para facilitar o treainemtno do modelo definimos uma função que transforma valores de entrada e saída, X e y, para o formato esperado do modelo de classificação.

In [23]:
def transform_to_model_input(X, y):
    encoded_sentences = encode_sentences(X, tokenizer, MAX_SEQUENCE_LENGTH)
    X_ = [encoded_sentences['input_ids'], encoded_sentences['attention_mask']]
    y_ = tf.convert_to_tensor(y.values, dtype=tf.int16)    
    return X_, y_

Abaixo é realizada a transformação dos dados de treinamento e validação para o formato adequado a ser usado pela função de treinamento.

In [29]:
X_train_, y_train_ = transform_to_model_input(X_train, y_train)
X_val_  , y_val_  = transform_to_model_input(X_val  , y_val)

## Etapa 5 - Treinamento do Modelo

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

NameError: ignored

In [31]:
log_dir='tensorboard_data/tb_bert'
model_save_path='bert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True)]

BATCH_SIZE = 32
history = []
history_ = model.fit(X_train_, y_train_, batch_size=BATCH_SIZE, epochs=1, validation_data=(X_val_,y_val_), callbacks=callbacks)
history.append(history_)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [32]:
history_ = model.fit(X_train_, y_train_, batch_size=BATCH_SIZE, epochs=2, validation_data=(X_val_,y_val_), callbacks=callbacks)
history.append(history_)

Epoch 1/2
443/443 [==============================] - 353s 797ms/step - loss: 0.3346 - accuracy: 0.8533 - val_loss: 0.3611 - val_accuracy: 0.8408
Epoch 2/2
443/443 [==============================] - 355s 802ms/step - loss: 0.2322 - accuracy: 0.9062 - val_loss: 0.3789 - val_accuracy: 0.8601


## Etapa 6 - Avaliação do Modelo

In [21]:
best_model = TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=num_labels, output_attentions=False, output_hidden_states=False)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
best_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_save_path='bert_model.h5'
best_model.load_weights(model_save_path)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
X_test_ , y_test_  = transform_to_model_input(X_test, y_test)

In [31]:
BATCH_SIZE=64
loss_test, acc_test = best_model.evaluate(X_test_, y_test_, batch_size=BATCH_SIZE)

111/111 [==============================] - 1069s 10s/step - loss: 0.3742 - accuracy: 0.8324


In [33]:
print("acc_test:", acc_test)

acc_test: 0.8324385285377502


In [26]:
df_fr = pd.read_csv('../data/app_reviews_fr_processed.csv')
df_fr = df_fr.dropna()
df_fr

,text,label,score
0,Je ne comprend absolument rien l'application e...,negative,1
1,"Arnaque, J'ai installée l'application sur mon ...",negative,1
2,L'appli crash dès que l'on essaie de rajouter ...,negative,1
3,L'application vous propose de fonctionner selo...,negative,1
4,Je ne trouve pas ANYDO si pratique ! Il faut p...,negative,1
...,...,...,...
9542,Easy very spécial,positive,5
9543,For lazy people.,positive,5
9544,Very nice planning app!,positive,5
9545,Tres bien comme appli mais quand on met un ren...,positive,5


In [28]:
df_fr['label'].replace(to_replace, inplace=True)
df_fr

,text,label,score
0,Je ne comprend absolument rien l'application e...,0,1
1,"Arnaque, J'ai installée l'application sur mon ...",0,1
2,L'appli crash dès que l'on essaie de rajouter ...,0,1
3,L'application vous propose de fonctionner selo...,0,1
4,Je ne trouve pas ANYDO si pratique ! Il faut p...,0,1
...,...,...,...
9542,Easy very spécial,1,5
9543,For lazy people.,1,5
9544,Very nice planning app!,1,5
9545,Tres bien comme appli mais quand on met un ren...,1,5


In [29]:
X_fr, y_fr = transform_to_model_input(df_fr['text'], df_fr['label'])

In [30]:
BATCH_SIZE=64
loss_test_fr, acc_test_fr = best_model.evaluate(X_fr, y_fr, batch_size=BATCH_SIZE)

 11/150 [=>............................] - ETA: 20:33 - loss: 0.8100 - accuracy: 0.6591

KeyboardInterrupt: 

# Conclusão